<a href="https://colab.research.google.com/github/tomonari-masada/course2021-nlp/blob/main/07_PyTorch_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch入門 (3)
* IMDbデータセットの感情分析をPyTorchを使っておこなう。
 * 前にscikit-learnを使って同じ作業をおこなった。
* 参考資料
 * PyTorch公式のチュートリアル https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
* データは以前作ったIMDbの文書埋め込みを使う。
* sentiment analysisのもっと高度な手法については、下記リンク先を参照。
 * https://github.com/bentrevett/pytorch-sentiment-analysis

## 1. fastTextによる文書埋め込みをMLPの入力として使うための準備
* MLP(多層パーセプトロン)の学習ぐらいは、空気を吸ったり吐いたりするぐらい自然にできるようにしておこう。

### データファイルが置いてあるGoogle Driveのパスを変数PATHに設定
* データファイルの扱い方
 * Blackboardで「自然言語処理特論」へ行く。
 * 「教材/課題/テスト」→「data」→「IMDb」と順にクリックする。
 * 見えている4つの「.npy」ファイルをダウンロードする。
 * ダウンロードした4つのファイルを、自分のGoogle Driveの適当な場所にアップロードする。
 * 次のセルで、その置き場所を指定する。

In [ ]:
PATH = '/content/drive/MyDrive/2021Courses/NLP/'

* （あらかじめランタイムのタイプをGPUに設定しておこう。）

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

np.random.seed(123)
torch.manual_seed(123)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!nvidia-smi

In [ ]:
device

### 単語埋め込みデータファイルの読み込み

In [ ]:
texts = dict()
labels = dict()
for tag in ['train', 'test']:
  with open(f'{PATH}{tag}.npy', 'rb') as f:
    texts[tag] = np.load(f)
  with open(f'{PATH}{tag}_labels.npy', 'rb') as f:
    labels[tag] = np.load(f)

In [ ]:
for tag in ['train', 'test']:
  print(texts[tag].shape)

### ndarrayをPyTorchのテンソルに変換

In [ ]:
for tag in ['train', 'test']:
  texts[tag], labels[tag] = torch.tensor(texts[tag]), torch.tensor(labels[tag])

## 2. 学習のための準備

### Datasetの利用

In [ ]:
from torch.utils.data import Dataset, random_split

class MyDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, index):
    return self.X[index], self.y[index]

train_valid = MyDataset(texts['train'], labels['train'])
test = MyDataset(texts['test'], labels['test'])

valid_size = len(train_valid) // 5
train_size = len(train_valid) - valid_size
train, valid = random_split(train_valid,
                            [train_size, valid_size],
                            generator=torch.Generator().manual_seed(42)
                            )

### DataLoaderの利用

In [ ]:
from torch.utils.data import DataLoader

# ミニバッチのサイズ
BATCH_SIZE = 100

# 訓練データだけシャッフル
train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid, batch_size=BATCH_SIZE)
test_loader = DataLoader(test, batch_size=BATCH_SIZE)

## 3. モデルの定義と学習の準備

### モデルの定義

In [ ]:
class TextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class):
    super(TextSentiment, self).__init__()
    self.fc1 = nn.Linear(embed_dim, 500)
    self.fc2 = nn.Linear(500, 100)
    self.fc3 = nn.Linear(100, num_class)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [ ]:
EMBED_DIM = texts['train'].size(1)
NUM_CLASS = len(np.unique(labels['train']))
model = TextSentiment(EMBED_DIM, NUM_CLASS).to(device)

In [ ]:
print(EMBED_DIM, NUM_CLASS)

### 損失関数とoptimizer

* 損失関数を除いて、以下の設定はいい加減なので、自分で調整してみよう。
* schedulerの使い方は、調べてみよう。

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
#scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20,50], gamma=0.1)

## 4. 分類器の訓練と評価

### 評価用の関数
* 正解率で評価する関数を定義しておく。

In [ ]:
def eval(model, criterion, loader):
  total_loss = 0.0
  total_acc = 0.0
  total_size = 0
  for input, target in loader:
    with torch.no_grad():
      input, target = input.to(device), target.to(device)
      output = model(input)
      loss = criterion(output, target)
      total_loss += loss.item() * len(target)
      total_acc += (output.argmax(1) == target).float().sum().item()
      total_size += len(target)

  return total_loss / total_size, total_acc / total_size

### 訓練用の関数
* ループの中身は前回とほぼ同じ。

In [ ]:
def train(model, criterion, optimizer, train_loader, valid_loader, n_epochs=100):
  # training loop
  for epoch in range(n_epochs):

    train_loss = 0.0
    for input, target in train_loader:
      output = model(input.to(device))
      loss = criterion(output, target.to(device))
      train_loss += loss.item() * len(target) # 表示用の集計

      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

    # valid loss
    valid_loss = 0
    with torch.no_grad():
      for input, target in valid_loader:
        output = model(input.to(device))
        loss = criterion(output, target.to(device))
        valid_loss += loss.item() * len(target)

    # logging
    print(f'epoch {epoch + 1:4d} |',
          f'train loss {train_loss / train_size:8.4f} |',
          f'valid loss {valid_loss / valid_size:8.4f}')

### 訓練と評価の実施

In [ ]:
train(model, criterion, optimizer, train_loader, valid_loader, 100)

In [ ]:
loss, acc = eval(model, criterion, train_loader)
print(f'train | loss : {loss:8.4f} ; accuracy : {acc:.3f}')

In [ ]:
loss, acc = eval(model, criterion, valid_loader)
print(f'valid | loss : {loss:8.4f} ; accuracy : {acc:.3f}')

# 課題7
* モデルやoptimizerやschedulerを変更して、validation setを使ってチューニングしよう。
* 最後に、自分で選択した設定を使って、test set上で評価しよう。

In [ ]:
loss, acc = eval(model, criterion, test_loader)
print(f'test | loss : {loss:8.4f} ; accuracy : {acc:.3f}')